In [1]:
import sys
import time
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import numpy as np
import matplotlib as plt
import cv2
import math
import os
import tensorflow as tf
from keras.layers import Conv2D, Conv2DTranspose, Dropout, BatchNormalization, Flatten, Reshape, Activation, Dense, UpSampling2D, LeakyReLU, Input
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop
from keras import backend as K

Using TensorFlow backend.


In [2]:
images=list()
for i in range(3000,6000):
    a=cv2.imread("img_align_celeba/%06d.jpg"%i,1)
    a=cv2.resize(a,(128,128))
    a=a/255.0
    a=a.tolist()
    images.append(a)
images=np.asarray(images)
np.save('data1.npy',images)

In [2]:
images = np.load('data.npy')

In [3]:
def gen(inputs, image_shape):
    l=32
    x=Dense(8 * 8 * 128)(inputs)
    x=Reshape((8, 8, 128))(x)
    
    x=UpSampling2D()(x)
    x=Conv2D(filters = 512, kernel_size=3, padding='same')(x)
    x=BatchNormalization()(x)
    #x=LeakyReLU(alpha=0.2)(x)
    x=Activation('relu')(x)
    
    x=UpSampling2D()(x)
    x=Conv2D(filters = 256, kernel_size=3, padding='same')(x)
    x=BatchNormalization(momentum=0.8)(x)
    #x=LeakyReLU(alpha=0.2)(x)
    x=Activation('relu')(x)
    
    x=UpSampling2D()(x)
    x=Conv2D(filters = 128, kernel_size=3, padding='same')(x)
    x=BatchNormalization(momentum=0.8)(x)
    #x=LeakyReLU(alpha=0.2)(x)
    x=Activation('relu')(x)
    
    x=UpSampling2D()(x)
    x=Conv2D(filters = 64, kernel_size=3, padding='same')(x)
    x=BatchNormalization(momentum=0.8)(x)
    #x=LeakyReLU(alpha=0.2)(x)
    x=Activation('relu')(x)

#    x=Conv2DTranspose(filters = 32, kernel_size=3, strides=1, padding='same', activation = 'sigmoid')(x)
#    x=BatchNormalization()(x)
    
    x=Conv2D(filters = 3, kernel_size=3, padding='same')(x)
#    x=BatchNormalization(momentum=0.8)(x)
    #x=LeakyReLU(alpha=0.2)(x)
    x=Activation('sigmoid')(x)
    
    generator=Model(inputs, x)
    return generator

In [4]:
def dis(inputs):
    x=inputs
    
    x= Conv2D(filters= 32, kernel_size=3, strides=2, padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)
    x=Dropout(0.25)(x)
    
    x= Conv2D(filters= 64, kernel_size=3, strides=2, padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)
    x=Dropout(0.25)(x)
    
    x= Conv2D(filters= 128, kernel_size=3, strides=2, padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)
    x=Dropout(0.25)(x)
    
    x= Conv2D(filters= 256, kernel_size=3, strides=1, padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)
    x=Dropout(0.25)(x)
    
    x= Conv2D(filters= 512, kernel_size=3, strides=1, padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)
    x=Dropout(0.25)(x)
    
    x=Flatten()(x)
    x=Dense(1, activation = 'sigmoid')(x)
    
    discriminator = Model(inputs, x)
    return discriminator

In [8]:
def models(latent_size):
    input_shape=(images.shape[1],images.shape[2],3)
    
    inputs = Input(shape=input_shape)
    discriminator=dis(inputs)
    discriminator.compile(loss = 'binary_crossentropy', optimizer=Adam(0.0002,0.5), metrics = ['accuracy'])
    
    inputs = Input(shape=(latent_size, ))
    generator=gen(inputs, input_shape)
    
    discriminator.trainable = False
    
    adversarial=Model(inputs, discriminator(generator(inputs)))
    adversarial.compile(loss='binary_crossentropy', optimizer = Adam(0.0002,0.1))
    
    models=(generator, discriminator, adversarial)
    return models

In [9]:
def train(models, epochs, batch_size, latent_size, save_interval=0, mod_save_interval=0):
    generator, discriminator, adversarial = models
    
    for i in range(epochs):
        t1=time.time()
        
        rand_indices = np.random.randint(0, images.shape[0], size = batch_size)
        real = images[rand_indices]
        noise = np.random.uniform(0, 1.0, size = [batch_size, latent_size])
        fake = generator.predict(noise)
        
        x = np.concatenate((real,fake))
        
        y = np.ones(batch_size*2)
        y[batch_size:]=0.0
        y=np.reshape(y,(batch_size*2,1))
        
        history = discriminator.train_on_batch(x,y)
        
        noise = np.random.uniform(0, 1.0, size = [batch_size, latent_size])
        y=np.ones(batch_size)
        y=np.reshape(y,(batch_size,1))
        
        """aloss, aacc =""" 
        history1 = adversarial.train_on_batch(noise, y)
        
        t2=time.time()
        
        print("Epoch : %d, [adversarial loss: %f] [discriminator loss: %f, acc: %f] time : %.4fs"%(i+1, history1, history[0], history[1], t2-t1))
        
        if save_interval!=0:
            if (i+1)%save_interval==0:
                noise = np.random.uniform(0, 1.0, size = [1, latent_size])
                n=generator.predict(noise)
                n=n[0]
                n*=255
                cv2.imwrite("%d.jpeg"%(i+1),n)
                print("Saved!")
        if mod_save_interval!=0:
            if (i+1)%mod_save_interval==0:
                generator.save('gen%d.h5'%(i+1))
                print("Generator Saved")

In [ ]:
model=models(100)
with tf.device('/gpu:0'):
    train(model, 3000, 64, 100, 10, 200)

Epoch : 1, [adversarial loss: 0.693470] [discriminator loss: 0.691826, acc: 0.523438] time : 19.3560s
Epoch : 2, [adversarial loss: 0.663280] [discriminator loss: 0.683778, acc: 0.695312] time : 16.2871s
Epoch : 3, [adversarial loss: 0.648797] [discriminator loss: 0.670600, acc: 0.718750] time : 16.2701s
Epoch : 4, [adversarial loss: 0.551598] [discriminator loss: 0.643435, acc: 0.585938] time : 16.4056s
Epoch : 5, [adversarial loss: 0.323880] [discriminator loss: 0.596677, acc: 0.718750] time : 16.3110s
Epoch : 6, [adversarial loss: 0.295862] [discriminator loss: 0.556324, acc: 0.859375] time : 16.4195s
Epoch : 7, [adversarial loss: 0.010790] [discriminator loss: 0.470104, acc: 0.718750] time : 16.3420s
Epoch : 8, [adversarial loss: 0.278159] [discriminator loss: 1.139541, acc: 0.500000] time : 16.3879s
Epoch : 9, [adversarial loss: 0.230732] [discriminator loss: 0.542096, acc: 0.625000] time : 16.4508s
Epoch : 10, [adversarial loss: 0.105974] [discriminator loss: 0.518085, acc: 0.640

Epoch : 81, [adversarial loss: 0.000000] [discriminator loss: 0.000287, acc: 1.000000] time : 16.2155s
Epoch : 82, [adversarial loss: 0.000000] [discriminator loss: 0.000255, acc: 1.000000] time : 16.1907s
Epoch : 83, [adversarial loss: 0.000000] [discriminator loss: 0.000197, acc: 1.000000] time : 16.2097s
Epoch : 84, [adversarial loss: 0.000000] [discriminator loss: 0.000201, acc: 1.000000] time : 16.1108s
Epoch : 85, [adversarial loss: 0.000000] [discriminator loss: 0.000230, acc: 1.000000] time : 16.1216s
Epoch : 86, [adversarial loss: 0.000000] [discriminator loss: 0.000123, acc: 1.000000] time : 16.2801s
Epoch : 87, [adversarial loss: 0.000000] [discriminator loss: 0.000127, acc: 1.000000] time : 16.1535s
Epoch : 88, [adversarial loss: 0.000000] [discriminator loss: 0.000135, acc: 1.000000] time : 16.2630s
Epoch : 89, [adversarial loss: 0.000000] [discriminator loss: 0.000122, acc: 1.000000] time : 16.1655s
Epoch : 90, [adversarial loss: 0.000000] [discriminator loss: 0.000120, a

Epoch : 160, [adversarial loss: 0.000000] [discriminator loss: 0.005284, acc: 1.000000] time : 16.1405s
Saved!
Epoch : 161, [adversarial loss: 0.000000] [discriminator loss: 0.001346, acc: 1.000000] time : 16.1542s
Epoch : 162, [adversarial loss: 0.000000] [discriminator loss: 0.000201, acc: 1.000000] time : 16.1254s
Epoch : 163, [adversarial loss: 0.000000] [discriminator loss: 0.000182, acc: 1.000000] time : 16.0844s
Epoch : 164, [adversarial loss: 0.000000] [discriminator loss: 0.000178, acc: 1.000000] time : 16.1457s
Epoch : 165, [adversarial loss: 0.000000] [discriminator loss: 0.000148, acc: 1.000000] time : 16.1482s
Epoch : 166, [adversarial loss: 0.000000] [discriminator loss: 0.003627, acc: 1.000000] time : 16.0824s
Epoch : 167, [adversarial loss: 0.000000] [discriminator loss: 0.003134, acc: 1.000000] time : 16.0329s
Epoch : 168, [adversarial loss: 0.000000] [discriminator loss: 0.002405, acc: 1.000000] time : 16.0823s
Epoch : 169, [adversarial loss: 0.000000] [discriminator 

Epoch : 239, [adversarial loss: 0.304363] [discriminator loss: 0.769036, acc: 0.242188] time : 16.3703s
Epoch : 240, [adversarial loss: 0.325160] [discriminator loss: 0.809124, acc: 0.234375] time : 16.3359s
Saved!
Epoch : 241, [adversarial loss: 0.341321] [discriminator loss: 0.755608, acc: 0.320312] time : 16.2616s
Epoch : 242, [adversarial loss: 0.326126] [discriminator loss: 0.733976, acc: 0.390625] time : 16.3652s
Epoch : 243, [adversarial loss: 0.305398] [discriminator loss: 0.753231, acc: 0.421875] time : 16.3552s
Epoch : 244, [adversarial loss: 0.345521] [discriminator loss: 0.734185, acc: 0.406250] time : 16.6361s
Epoch : 245, [adversarial loss: 0.550732] [discriminator loss: 0.710892, acc: 0.460938] time : 16.3969s
Epoch : 246, [adversarial loss: 0.510562] [discriminator loss: 0.675752, acc: 0.476562] time : 16.2437s
Epoch : 247, [adversarial loss: 0.420317] [discriminator loss: 0.691670, acc: 0.515625] time : 16.3494s
Epoch : 248, [adversarial loss: 0.434701] [discriminator 

Epoch : 318, [adversarial loss: 0.628814] [discriminator loss: 0.630830, acc: 0.773438] time : 16.3771s
Epoch : 319, [adversarial loss: 0.519586] [discriminator loss: 0.722630, acc: 0.296875] time : 16.3266s
Epoch : 320, [adversarial loss: 0.513274] [discriminator loss: 0.796287, acc: 0.320312] time : 16.2951s
Saved!
Epoch : 321, [adversarial loss: 0.521845] [discriminator loss: 0.815668, acc: 0.218750] time : 16.2773s
Epoch : 322, [adversarial loss: 0.482694] [discriminator loss: 0.777826, acc: 0.187500] time : 16.3058s
Epoch : 323, [adversarial loss: 0.500185] [discriminator loss: 0.714765, acc: 0.343750] time : 16.3527s
Epoch : 324, [adversarial loss: 0.546421] [discriminator loss: 0.683836, acc: 0.398438] time : 16.2812s
Epoch : 325, [adversarial loss: 0.591672] [discriminator loss: 0.666233, acc: 0.414062] time : 16.2452s
Epoch : 326, [adversarial loss: 0.598350] [discriminator loss: 0.695793, acc: 0.429688] time : 16.3550s
Epoch : 327, [adversarial loss: 0.586370] [discriminator 

Epoch : 397, [adversarial loss: 0.570142] [discriminator loss: 0.692200, acc: 0.390625] time : 16.2919s
Epoch : 398, [adversarial loss: 0.587122] [discriminator loss: 0.682814, acc: 0.429688] time : 16.9427s
Epoch : 399, [adversarial loss: 0.606910] [discriminator loss: 0.696715, acc: 0.375000] time : 16.3363s
Epoch : 400, [adversarial loss: 0.627405] [discriminator loss: 0.690253, acc: 0.382812] time : 16.8550s
Saved!
Generator Saved
Epoch : 401, [adversarial loss: 0.648986] [discriminator loss: 0.695287, acc: 0.296875] time : 16.8458s
Epoch : 402, [adversarial loss: 0.674982] [discriminator loss: 0.686825, acc: 0.421875] time : 16.4205s
Epoch : 403, [adversarial loss: 0.687295] [discriminator loss: 0.681522, acc: 0.601562] time : 16.3429s
Epoch : 404, [adversarial loss: 0.697296] [discriminator loss: 0.680772, acc: 0.695312] time : 16.3439s
Epoch : 405, [adversarial loss: 0.705784] [discriminator loss: 0.679902, acc: 0.710938] time : 16.3701s
Epoch : 406, [adversarial loss: 0.699337]

In [ ]:
generator, discriminator, adversarial = model
generator.save('generator.h5')
discriminator.save('discriminator.h5')
adversarial.save('adversarial.h5')

In [13]:
gener=load_model('generator.h5')
discr=load_model('discriminator.h5')
adver=load_model('adversarial.h5')
model=(gener,discr,adver)
with tf.device('/gpu:0'):
    train(model, 3000, 64, 100, 10, 200)

Epoch : 1, [adversarial loss: 0.000000] [discriminator loss: 0.000849, acc: 1.000000] time : 16.2842s
Epoch : 2, [adversarial loss: 0.000000] [discriminator loss: 0.000422, acc: 1.000000] time : 15.4231s
Epoch : 3, [adversarial loss: 0.000000] [discriminator loss: 0.000773, acc: 1.000000] time : 15.6071s
Epoch : 4, [adversarial loss: 0.000000] [discriminator loss: 0.000336, acc: 1.000000] time : 15.6962s
Epoch : 5, [adversarial loss: 0.000000] [discriminator loss: 0.000388, acc: 1.000000] time : 15.9063s
Epoch : 6, [adversarial loss: 0.000000] [discriminator loss: 0.000285, acc: 1.000000] time : 15.8788s
Epoch : 7, [adversarial loss: 0.000000] [discriminator loss: 0.000283, acc: 1.000000] time : 15.9194s
Epoch : 8, [adversarial loss: 0.000000] [discriminator loss: 0.000214, acc: 1.000000] time : 15.9824s
Epoch : 9, [adversarial loss: 0.000000] [discriminator loss: 0.000301, acc: 1.000000] time : 15.9758s
Epoch : 10, [adversarial loss: 0.000000] [discriminator loss: 0.000172, acc: 1.000

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()
    print(devices)